Modelo de Regresión Logistica en RapidMiner
===========================================

Visto en el articulo anterior, analizamos los datos disponibles en los conjuntos de datos de enfermedad cardíaca y, a partir de nuestras observaciones, decidimos que la regresión logística sería un buen algoritmo para usar en este caso dado que la variable objetivo es binaria (0 o 1). En este artículo, veremos cómo implementar la regresión logística en RapidMiner.

## Cargar datos

Anteriormente, utilizamos pandas para combinar los conjuntos de datos en uno solo, y mejor formatearlo para su importacion en RapidMiner. La carga es requiere importar el archivo full.csv creado en el articulo anterior. En el proceso, verificamos que los datos tengan los tipos correctos, y corregimos basados en los tipos de datos vistos en el articulo anterior. Luego de importado, introducimos el conjunto de datos al proceso de RapidMiner.

## Preprocesamiento

Primera tarea de preprocesamiento que realizaremos, es la de convertir la variable de salida en un atributo binario. Para ello, utilizaremos el operador `Replace`, que nos permite reemplazar valores en un atributo. En este caso, reemplazaremos todo valor de la columna `num` que sea mayor a 0, por el valor 1. Tambien nos aseguraremos de asignar a este atributo el rol de `label` utilizando `Set Role`, y convertirlo en un atributo binario utilizando `Nominal to Binominal`.

Luego, inspeccionemos los casos de atributos con valores faltantes. Para ello, revisamos la estadística del conjunto resultante. Aquí podemos ver que existen varios atributos con valores faltantes, la cantidad de estos variando entre muy pocos y casi todo el conjunto de los casos. Tomamos nota de estos atributos, decidiendo que los atributos con más de 400 valores faltantes serán eliminados. Estos atributos son:

- ca
- cathef
- cigs
- diag
- dm
- earlobe
- exerckm
- exeref
- exerwm
- famhist
- junk
- om2
- pncaden
- ramus
- restckm
- restef
- restwm
- rldv5
- smoke
- thal
- thalpul
- thalsev
- thaltime
- years

Utilizamos el operador `Select Attributes` para eliminar estos atributos del conjunto de datos.

Siguiendo buscando atributos que no aporten a la solucion del problema, nos encontramos con la documentación de los conjuntos de datos, donde se especifica la definición de estos. En esta documentación, se especifica que varios atributos no son utilizados, no estan bien definidos, o no aportan al problema. Estos atributos son:

- ccf
- dummy
- lvf
- lvx1
- lvx2
- lvx3
- lvx4
- name

Nuevamente utilizamos el operador `Select Attributes` para eliminar estos atributos del conjunto de datos. Por el momento, nos quedamos con los atributos restantes, y continuamos con el preprocesamiento de los datos.

Pasamos a reemplazar los valores faltantes de los atributos restantes. Para ello, utilizamos el operador `Replace Missing Values` para reemplazar los valores faltantes de los atributos restantes. En este caso, utilizaremos el valor promedio de cada atributo. Esto solo lo haremos para los atributos numéricos reales, ya que en los atributos categóricos, el valor promedio no tiene sentido. Para los atributos categóricos, utilizaremos el valor más frecuente para reemplazar los valores faltantes. En RapidMiner, esto requiere seleccionar el valor de reemplazo como `average`, que en atributos nominales se convierte en la moda del atributo. Utilizaremos dos operadores `Replace Missing Values`, uno para los atributos numéricos reales, y otro para los atributos categóricos. Ambos utilizando el valor `average` como valor de reemplazo.

El siguiente paso es remover los atributos correlacionados. RapidMiner nos permite hacer esto utilizando el operador `Remove Correlated Attributes`. En este caso, utilizaremos un umbral de 0.35 para remover atributos de correlación significativa. 

Volvemos ahora a revisar los atributos a utilizar. Para optimizar esta selección, utilizaremos el operador `Select Attributes` para seleccionar los 20 atributos más relevantes para el modelo. Esto lo haremos dentro de un `Cross Validation` de 10 folds, para prevenir contaminación accidental de los datos en la selección de atributos. Dentro de este operador, utilizaremos el operador `Logistic Regression` para evaluar el la seleccion de atributos dentro de un `Split Validation` de 70% para entrenamiento y 30% para prueba. 

Como ultimo paso en el preprocesamiento de los datos, utilizaremos el operador `Optimize Selection` utilizando dirección `forwards` para seleccionar los atributos más relevantes para el modelo, y filtraremos cualquier ejemplo restante que tenga valores nulos utilizando `Filter Examples`. Esto lo haremos dentro del propio operador de `Cross Validation` utilizado para entrenar el modelo como veremos en la siguiente sección. Esto es para prevenir contaminación accidental de los datos en la selección de atributos.

## Entrenamiento

Para la etapa de entrenamiento, utilizaremos el operador `Logistic Regression` para entrenar el modelo. En este caso, utilizaremos un `Cross Validation` de 10 folds para evaluar el modelo, y una normalización de los datos utilizando el metodo `z-normalization` y el operador `Normalize` previo a entrenar el modelo para los datos de entrenamiento, y previa a probar el modelo para los datos de prueba.

El modelo de preprocesamiento creado por el operador `Normalize` es utilizado luego en la etapa de prueba para normalizar los datos de prueba. La aplicación tanto del modelo de preprocesamiento como del modelo predictivo en la etapa de prueba se realiza utilizando 2 operadores `Apply Model`.

## Proceso Completo

El proceso completo se puede ver en las siguientes imagenes:

![Proceso completo 01](../images/rm-rl-01.png)
![Proceso completo 02](../images/rm-rl-02.png)
![Proceso completo 03](../images/rm-rl-03.png)
![Proceso completo 04](../images/rm-rl-04.png)

## Evaluación

Una vez ejecutado el proceso, podemos ver los resultados de la evaluación del modelo en la pestaña `Results`. Aquí podemos ver la precisión del modelo y su matriz de confusión en la salida de `Performance Vector`:

![Evaluación 01](../images/rm-rl-05.png)

El resultado es una precisión del 91.63%. Considero que este es un buen resultado, dado la mala calidad de los datos, con muchos valores faltantes y atributos no relevantes.



